In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
rus= RandomUnderSampler(random_state= 15)

X_rus, y_rus= rus.fit_resample(X, y)
print(f'Random Under-Sampling: {Counter(y_rus)}')

Random Under-Sampling: Counter({0: 4561, 1: 4561})


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

X_rus_train, y_rus_train= rus.fit_resample(X_train, y_train)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr_rus= LogisticRegression(random_state=15)
lr_rus.fit(X_rus_train, y_rus_train)
y_pred_lr_rus= lr_rus.predict(X_test)
y_pred_prob_lr_rus= lr_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_rus, labels= lr_rus.classes_)
print(classification_report(y_test, y_pred_lr_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr_rus))

              precision    recall  f1-score   support

           0       0.97      0.74      0.84      8517
           1       0.25      0.81      0.38       912

    accuracy                           0.74      9429
   macro avg       0.61      0.77      0.61      9429
weighted avg       0.90      0.74      0.79      9429

Precision: 0.24757443961191034
Accuracy: 0.7432389436843779
AUC: 0.8538143012221172


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_rus= DecisionTreeClassifier(random_state=15)
dt_rus.fit(X_rus_train, y_rus_train)
y_pred_dt_rus= dt_rus.predict(X_test)
y_pred_prob_dt_rus= dt_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_rus, labels= dt_rus.classes_)
print(classification_report(y_test, y_pred_dt_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_rus))

              precision    recall  f1-score   support

           0       0.95      0.65      0.77      8517
           1       0.17      0.66      0.27       912

    accuracy                           0.65      9429
   macro avg       0.56      0.66      0.52      9429
weighted avg       0.87      0.65      0.72      9429

Precision: 0.16887510572314632
Accuracy: 0.654152083996182
AUC: 0.6552327491559709


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_rus= GridSearchCV(estimator= dt_rus, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_rus.fit(X_rus_train, y_rus_train)

print("Best Parameters:", gs_dt_rus.best_params_)
print("Best Precision Score:", gs_dt_rus.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 20}
Best Precision Score: 0.7219336344792142


In [12]:
tuned_dt_rus= gs_dt_rus.best_estimator_
y_pred_tuned_dt_rus= tuned_dt_rus.predict(X_test)
y_pred_prob_tuned_dt_rus= tuned_dt_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_rus, labels= tuned_dt_rus.classes_)
print(classification_report(y_test, y_pred_tuned_dt_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_rus))

              precision    recall  f1-score   support

           0       0.96      0.71      0.82      8517
           1       0.22      0.74      0.34       912

    accuracy                           0.72      9429
   macro avg       0.59      0.73      0.58      9429
weighted avg       0.89      0.72      0.77      9429

Precision: 0.21743325828240592
Accuracy: 0.7169371089192915
AUC: 0.7950726513948367


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_rus= KNeighborsClassifier()
knn_rus.fit(X_rus_train, y_rus_train)
y_pred_knn_rus= knn_rus.predict(X_test)
y_pred_prob_knn_rus= knn_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_rus, labels= knn_rus.classes_)
print(classification_report(y_test, y_pred_knn_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_rus))

              precision    recall  f1-score   support

           0       0.96      0.70      0.81      8517
           1       0.21      0.72      0.32       912

    accuracy                           0.70      9429
   macro avg       0.58      0.71      0.57      9429
weighted avg       0.89      0.70      0.76      9429

Precision: 0.20586400499064256
Accuracy: 0.7032559126100328
AUC: 0.776287852571431


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_rus= GridSearchCV(estimator= knn_rus, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_rus.fit(X_rus_train, y_rus_train)
print("Best Parameters:", gs_knn_rus.best_params_)
print("Best Precision Score:", gs_knn_rus.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 8, 'weights': 'uniform'}
Best Precision Score: 0.7512228612778743


In [15]:
tuned_knn_rus= gs_knn_rus.best_estimator_
y_pred_tuned_knn_rus= tuned_knn_rus.predict(X_test)
y_pred_prob_tuned_knn_rus= tuned_knn_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_rus, labels= tuned_knn_rus.classes_)
print(classification_report(y_test, y_pred_tuned_knn_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_rus))

              precision    recall  f1-score   support

           0       0.96      0.75      0.84      8517
           1       0.23      0.68      0.34       912

    accuracy                           0.75      9429
   macro avg       0.59      0.72      0.59      9429
weighted avg       0.89      0.75      0.79      9429

Precision: 0.22747252747252747
Accuracy: 0.7454661151765829
AUC: 0.7977052860223826


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_rus= GaussianNB()
nb_rus.fit(X_rus_train, y_rus_train)
y_pred_nb_rus= nb_rus.predict(X_test)
y_pred_prob_nb_rus= nb_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_rus, labels= nb_rus.classes_)
print(classification_report(y_test, y_pred_nb_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_rus))

              precision    recall  f1-score   support

           0       0.98      0.59      0.74      8517
           1       0.18      0.86      0.30       912

    accuracy                           0.62      9429
   macro avg       0.58      0.73      0.52      9429
weighted avg       0.90      0.62      0.69      9429

Precision: 0.1840074818798223
Accuracy: 0.616608335984728
AUC: 0.8059937915706257


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_rus= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_rus.fit(X_rus_train, y_rus_train)
y_pred_svc_rus= svc_rus.predict(X_test)
y_pred_prob_svc_rus= svc_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_rus, labels= svc_rus.classes_)
print(classification_report(y_test, y_pred_svc_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_rus))

              precision    recall  f1-score   support

           0       0.98      0.17      0.29      8517
           1       0.11      0.96      0.20       912

    accuracy                           0.25      9429
   macro avg       0.54      0.57      0.24      9429
weighted avg       0.89      0.25      0.28      9429

Precision: 0.11044025157232705
Accuracy: 0.24636758935199915
AUC: 0.6986840946589793


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_rus= RandomForestClassifier(random_state=15)
rf_rus.fit(X_rus_train, y_rus_train)
y_pred_rf_rus= rf_rus.predict(X_test)
y_pred_prob_rf_rus= rf_rus.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_rus, labels= rf_rus.classes_)
print(classification_report(y_test, y_pred_rf_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_rus))

              precision    recall  f1-score   support

           0       0.97      0.70      0.82      8517
           1       0.23      0.82      0.36       912

    accuracy                           0.71      9429
   macro avg       0.60      0.76      0.59      9429
weighted avg       0.90      0.71      0.77      9429

Precision: 0.22794117647058823
Accuracy: 0.7149220489977728
AUC: 0.831484605608185


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_rus= GridSearchCV(estimator= rf_rus, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_rus.fit(X_rus_train, y_rus_train)
print("Best Parameters:", gs_tuned_rf_rus.best_params_)
print("Best Precision Score:", gs_tuned_rf_rus.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Best Precision Score: 0.7372909750396673


In [21]:
tuned_rf_rus= gs_tuned_rf_rus.best_estimator_
y_pred_tuned_rf_rus= tuned_rf_rus.predict(X_test)
y_pred_prob_tuned_rf_rus= tuned_rf_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_rus, labels= tuned_rf_rus.classes_)
print(classification_report(y_test, y_pred_tuned_rf_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_rus))

              precision    recall  f1-score   support

           0       0.98      0.71      0.82      8517
           1       0.24      0.84      0.37       912

    accuracy                           0.72      9429
   macro avg       0.61      0.78      0.60      9429
weighted avg       0.90      0.72      0.78      9429

Precision: 0.23636363636363636
Accuracy: 0.7218156750450737
AUC: 0.8472304616772647


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_rus= AdaBoostClassifier(random_state=15)
ada_rus.fit(X_rus_train, y_rus_train)
y_pred_ada_rus= ada_rus.predict(X_test)
y_pred_prob_ada_rus= ada_rus.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_rus, labels= ada_rus.classes_)
print(classification_report(y_test, y_pred_ada_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_rus))

              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.26      0.81      0.39       912

    accuracy                           0.75      9429
   macro avg       0.61      0.78      0.62      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.25624133148404993
Accuracy: 0.7541626895747163
AUC: 0.8540979187136564


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_rus= GridSearchCV(estimator= ada_rus, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_rus.fit(X_rus_train, y_rus_train)
print("Best Parameters:", gs_ada_rus.best_params_)
print("Best Precision Score:", gs_ada_rus.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 50}
Best Precision Score: 0.7528307660879495


In [25]:
tuned_ada_rus= gs_ada_rus.best_estimator_
y_pred_tuned_ada_rus= tuned_ada_rus.predict(X_test)
y_pred_prob_tuned_ada_rus= tuned_ada_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_rus, labels= tuned_ada_rus.classes_)
print(classification_report(y_test, y_pred_tuned_ada_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_rus))

              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.26      0.81      0.39       912

    accuracy                           0.75      9429
   macro avg       0.61      0.78      0.62      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.25624133148404993
Accuracy: 0.7541626895747163
AUC: 0.8540979187136564


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_rus= GradientBoostingClassifier(random_state=15)
grb_rus.fit(X_rus_train, y_rus_train)
y_pred_grb_rus= grb_rus.predict(X_test)
y_pred_prob_grb_rus= grb_rus.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_rus, labels= grb_rus.classes_)
print(classification_report(y_test, y_pred_grb_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_rus))

              precision    recall  f1-score   support

           0       0.97      0.73      0.84      8517
           1       0.25      0.82      0.38       912

    accuracy                           0.74      9429
   macro avg       0.61      0.78      0.61      9429
weighted avg       0.90      0.74      0.79      9429

Precision: 0.24842245101295252
Accuracy: 0.7426026089723194
AUC: 0.8512022652321775


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_rus= GridSearchCV(estimator= grb_rus, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_rus.fit(X_rus_train, y_rus_train)

print("Best Parameters:", gs_grb_rus.best_params_)
print("Best Precision Score:", gs_grb_rus.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.7416740757615884


In [28]:
tuned_grb_rus= gs_grb_rus.best_estimator_
y_pred_tuned_grb_rus= tuned_grb_rus.predict(X_test)
y_pred_prob_tuned_grb_rus= tuned_grb_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_rus, labels= tuned_grb_rus.classes_)
print(classification_report(y_test, y_pred_tuned_grb_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_rus))

              precision    recall  f1-score   support

           0       0.97      0.73      0.84      8517
           1       0.25      0.82      0.38       912

    accuracy                           0.74      9429
   macro avg       0.61      0.78      0.61      9429
weighted avg       0.90      0.74      0.79      9429

Precision: 0.24842245101295252
Accuracy: 0.7426026089723194
AUC: 0.8512022652321775


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_rus= XGBClassifier(random_state=15)
xgb_rus.fit(X_rus_train, y_rus_train)
y_pred_xgb_rus= xgb_rus.predict(X_test)
y_pred_prob_xgb_rus= xgb_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_rus, labels= xgb_rus.classes_)
print(classification_report(y_test, y_pred_xgb_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_rus))

              precision    recall  f1-score   support

           0       0.97      0.71      0.82      8517
           1       0.23      0.79      0.35       912

    accuracy                           0.72      9429
   macro avg       0.60      0.75      0.59      9429
weighted avg       0.90      0.72      0.78      9429

Precision: 0.22810913705583757
Accuracy: 0.7214975076890444
AUC: 0.8224834515695132


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_rus= GridSearchCV(estimator= xgb_rus, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_rus.fit(X_rus_train, y_rus_train)

print("Best Parameters:", gs_xgb_rus.best_params_)
print("Best Precision Score:", gs_xgb_rus.best_score_)

tuned_xgb_rus= gs_xgb_rus.best_estimator_
y_pred_tuned_xgb_rus= tuned_xgb_rus.predict(X_test)
y_pred_prob_tuned_xgb_rus= tuned_xgb_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_rus, labels= tuned_xgb_rus.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_rus))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.7347532239411896
              precision    recall  f1-score   support

           0       0.97      0.72      0.83      8517
           1       0.24      0.81      0.37       912

    accuracy                           0.73      9429
   macro avg       0.60      0.77      0.60      9429
weighted avg       0.90      0.73      0.78      9429

Precision: 0.23630573248407644
Accuracy: 0.7276487432389437
AUC: 0.842692710554124


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_rus= LGBMClassifier(random_state=15)
lgm_rus.fit(X_rus_train, y_rus_train)
y_pred_lgm_rus= lgm_rus.predict(X_test)
y_pred_prob_lgm_rus= lgm_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_rus, labels= lgm_rus.classes_)
print(classification_report(y_test, y_pred_lgm_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_rus))

[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.135829 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 222
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.97      0.72      0.82      8517
           1       0.23      0.81      0.36       912

    accuracy                           0.72      9429
   macro avg       0.60      0.76      0.59      9429
weighted avg       0.90      0.72      0.78      9429

Precision: 0.23419721871049304
Accuracy: 0.7248912928200233
AUC: 0.843141117146512


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_rus= GridSearchCV(estimator= lgm_rus, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_rus.fit(X_rus_train, y_rus_train)

print("Best Parameters:", gs_lgm_rus.best_params_)
print("Best Precision Score:", gs_lgm_rus.best_score_)

tuned_lgm_rus= gs_lgm_rus.best_estimator_
y_pred_tuned_lgm_rus= tuned_lgm_rus.predict(X_test)
y_pred_prob_tuned_lgm_rus= tuned_lgm_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_rus, labels= tuned_lgm_rus.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_rus))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Info] Number of positive: 3649, number of negative: 3649
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006606 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 195
[LightGBM] [Info] Number of data points in the train set: 7298, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 400, 'num_leaves': 31, 'reg_alpha': 0.5}
Best Precision Score: 0.7391396247055961
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
              precision    recall  f1-score   support

           0       0.97      0.72      0.83      8517
           1       0.24      0.83      0.37       912

    accuracy                           0.73      9429
   macro avg       0.61      0.77      0.60      9429
weighted avg       0.90      0.73      0.79      9429

Precision: 0.24104859335038364
Accuracy: 0.7314667515112949
AUC: 0.8494195818888538


--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_rus= CatBoostClassifier(random_state=15)
cat_rus.fit(X_rus_train, y_rus_train)
y_pred_cat_rus= cat_rus.predict(X_test)
y_pred_prob_cat_rus= cat_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_rus, labels= cat_rus.classes_)
print(classification_report(y_test, y_pred_cat_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_rus))

Learning rate set to 0.024073
0:	learn: 0.6854679	total: 144ms	remaining: 2m 24s
1:	learn: 0.6779882	total: 151ms	remaining: 1m 15s
2:	learn: 0.6700759	total: 161ms	remaining: 53.4s
3:	learn: 0.6637136	total: 170ms	remaining: 42.2s
4:	learn: 0.6566016	total: 180ms	remaining: 35.9s
5:	learn: 0.6505402	total: 190ms	remaining: 31.5s
6:	learn: 0.6443765	total: 197ms	remaining: 28s
7:	learn: 0.6390451	total: 204ms	remaining: 25.2s
8:	learn: 0.6336233	total: 210ms	remaining: 23.1s
9:	learn: 0.6283714	total: 221ms	remaining: 21.9s
10:	learn: 0.6230929	total: 229ms	remaining: 20.6s
11:	learn: 0.6182893	total: 236ms	remaining: 19.4s
12:	learn: 0.6143513	total: 244ms	remaining: 18.5s
13:	learn: 0.6098455	total: 251ms	remaining: 17.7s
14:	learn: 0.6060126	total: 258ms	remaining: 16.9s
15:	learn: 0.6018309	total: 265ms	remaining: 16.3s
16:	learn: 0.5982015	total: 272ms	remaining: 15.8s
17:	learn: 0.5950998	total: 279ms	remaining: 15.2s
18:	learn: 0.5916409	total: 287ms	remaining: 14.8s
19:	learn: 

163:	learn: 0.4727270	total: 1.48s	remaining: 7.52s
164:	learn: 0.4724509	total: 1.49s	remaining: 7.52s
165:	learn: 0.4721972	total: 1.5s	remaining: 7.51s
166:	learn: 0.4720118	total: 1.5s	remaining: 7.49s
167:	learn: 0.4718320	total: 1.51s	remaining: 7.49s
168:	learn: 0.4716320	total: 1.52s	remaining: 7.47s
169:	learn: 0.4715049	total: 1.53s	remaining: 7.46s
170:	learn: 0.4713182	total: 1.54s	remaining: 7.45s
171:	learn: 0.4711538	total: 1.54s	remaining: 7.44s
172:	learn: 0.4709262	total: 1.55s	remaining: 7.43s
173:	learn: 0.4706893	total: 1.56s	remaining: 7.42s
174:	learn: 0.4705128	total: 1.57s	remaining: 7.4s
175:	learn: 0.4703113	total: 1.57s	remaining: 7.38s
176:	learn: 0.4700220	total: 1.58s	remaining: 7.36s
177:	learn: 0.4697946	total: 1.59s	remaining: 7.34s
178:	learn: 0.4695672	total: 1.6s	remaining: 7.32s
179:	learn: 0.4693939	total: 1.6s	remaining: 7.31s
180:	learn: 0.4692408	total: 1.61s	remaining: 7.29s
181:	learn: 0.4690831	total: 1.62s	remaining: 7.28s
182:	learn: 0.468

329:	learn: 0.4436634	total: 2.76s	remaining: 5.61s
330:	learn: 0.4434250	total: 2.77s	remaining: 5.61s
331:	learn: 0.4433019	total: 2.78s	remaining: 5.6s
332:	learn: 0.4432297	total: 2.79s	remaining: 5.58s
333:	learn: 0.4430614	total: 2.79s	remaining: 5.57s
334:	learn: 0.4428921	total: 2.8s	remaining: 5.56s
335:	learn: 0.4427008	total: 2.81s	remaining: 5.55s
336:	learn: 0.4426428	total: 2.82s	remaining: 5.54s
337:	learn: 0.4424567	total: 2.82s	remaining: 5.53s
338:	learn: 0.4422626	total: 2.83s	remaining: 5.52s
339:	learn: 0.4420910	total: 2.84s	remaining: 5.51s
340:	learn: 0.4420081	total: 2.85s	remaining: 5.5s
341:	learn: 0.4418710	total: 2.85s	remaining: 5.49s
342:	learn: 0.4417241	total: 2.86s	remaining: 5.48s
343:	learn: 0.4416007	total: 2.87s	remaining: 5.48s
344:	learn: 0.4414374	total: 2.88s	remaining: 5.47s
345:	learn: 0.4412606	total: 2.89s	remaining: 5.46s
346:	learn: 0.4411018	total: 2.9s	remaining: 5.45s
347:	learn: 0.4410160	total: 2.9s	remaining: 5.44s
348:	learn: 0.440

492:	learn: 0.4164398	total: 4.07s	remaining: 4.18s
493:	learn: 0.4162742	total: 4.08s	remaining: 4.18s
494:	learn: 0.4161346	total: 4.08s	remaining: 4.17s
495:	learn: 0.4159792	total: 4.1s	remaining: 4.16s
496:	learn: 0.4157962	total: 4.1s	remaining: 4.15s
497:	learn: 0.4156167	total: 4.11s	remaining: 4.14s
498:	learn: 0.4154525	total: 4.12s	remaining: 4.13s
499:	learn: 0.4152495	total: 4.13s	remaining: 4.13s
500:	learn: 0.4150368	total: 4.13s	remaining: 4.12s
501:	learn: 0.4148693	total: 4.14s	remaining: 4.11s
502:	learn: 0.4146254	total: 4.15s	remaining: 4.1s
503:	learn: 0.4145208	total: 4.16s	remaining: 4.09s
504:	learn: 0.4143381	total: 4.16s	remaining: 4.08s
505:	learn: 0.4141256	total: 4.17s	remaining: 4.07s
506:	learn: 0.4139272	total: 4.18s	remaining: 4.06s
507:	learn: 0.4137656	total: 4.19s	remaining: 4.05s
508:	learn: 0.4136469	total: 4.2s	remaining: 4.05s
509:	learn: 0.4135058	total: 4.21s	remaining: 4.04s
510:	learn: 0.4133267	total: 4.21s	remaining: 4.03s
511:	learn: 0.41

653:	learn: 0.3926911	total: 5.33s	remaining: 2.82s
654:	learn: 0.3925452	total: 5.34s	remaining: 2.81s
655:	learn: 0.3923899	total: 5.35s	remaining: 2.8s
656:	learn: 0.3923283	total: 5.36s	remaining: 2.79s
657:	learn: 0.3921821	total: 5.36s	remaining: 2.79s
658:	learn: 0.3920748	total: 5.37s	remaining: 2.78s
659:	learn: 0.3919343	total: 5.38s	remaining: 2.77s
660:	learn: 0.3917989	total: 5.39s	remaining: 2.76s
661:	learn: 0.3917355	total: 5.39s	remaining: 2.75s
662:	learn: 0.3916365	total: 5.4s	remaining: 2.75s
663:	learn: 0.3915179	total: 5.41s	remaining: 2.74s
664:	learn: 0.3913119	total: 5.42s	remaining: 2.73s
665:	learn: 0.3911779	total: 5.42s	remaining: 2.72s
666:	learn: 0.3911325	total: 5.43s	remaining: 2.71s
667:	learn: 0.3910529	total: 5.44s	remaining: 2.7s
668:	learn: 0.3909298	total: 5.45s	remaining: 2.7s
669:	learn: 0.3908589	total: 5.46s	remaining: 2.69s
670:	learn: 0.3906460	total: 5.47s	remaining: 2.68s
671:	learn: 0.3905311	total: 5.47s	remaining: 2.67s
672:	learn: 0.39

815:	learn: 0.3739503	total: 6.58s	remaining: 1.48s
816:	learn: 0.3738299	total: 6.59s	remaining: 1.48s
817:	learn: 0.3736651	total: 6.6s	remaining: 1.47s
818:	learn: 0.3736016	total: 6.6s	remaining: 1.46s
819:	learn: 0.3735254	total: 6.61s	remaining: 1.45s
820:	learn: 0.3734589	total: 6.62s	remaining: 1.44s
821:	learn: 0.3733685	total: 6.63s	remaining: 1.43s
822:	learn: 0.3732674	total: 6.63s	remaining: 1.43s
823:	learn: 0.3732140	total: 6.64s	remaining: 1.42s
824:	learn: 0.3731509	total: 6.65s	remaining: 1.41s
825:	learn: 0.3730252	total: 6.66s	remaining: 1.4s
826:	learn: 0.3728970	total: 6.66s	remaining: 1.39s
827:	learn: 0.3728206	total: 6.67s	remaining: 1.39s
828:	learn: 0.3726780	total: 6.67s	remaining: 1.38s
829:	learn: 0.3726172	total: 6.68s	remaining: 1.37s
830:	learn: 0.3725005	total: 6.69s	remaining: 1.36s
831:	learn: 0.3724021	total: 6.7s	remaining: 1.35s
832:	learn: 0.3723170	total: 6.71s	remaining: 1.34s
833:	learn: 0.3722182	total: 6.71s	remaining: 1.34s
834:	learn: 0.37

990:	learn: 0.3557910	total: 8.03s	remaining: 73ms
991:	learn: 0.3557076	total: 8.04s	remaining: 64.9ms
992:	learn: 0.3556294	total: 8.05s	remaining: 56.8ms
993:	learn: 0.3555547	total: 8.06s	remaining: 48.6ms
994:	learn: 0.3554406	total: 8.06s	remaining: 40.5ms
995:	learn: 0.3553763	total: 8.07s	remaining: 32.4ms
996:	learn: 0.3553096	total: 8.08s	remaining: 24.3ms
997:	learn: 0.3551501	total: 8.09s	remaining: 16.2ms
998:	learn: 0.3550750	total: 8.1s	remaining: 8.1ms
999:	learn: 0.3549387	total: 8.1s	remaining: 0us
              precision    recall  f1-score   support

           0       0.97      0.72      0.83      8517
           1       0.24      0.82      0.37       912

    accuracy                           0.73      9429
   macro avg       0.61      0.77      0.60      9429
weighted avg       0.90      0.73      0.78      9429

Precision: 0.23756377551020408
Accuracy: 0.7287093010923746
AUC: 0.8447348079897996


In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_rus= GridSearchCV(estimator= cat_rus, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_rus.fit(X_rus_train, y_rus_train)

print("Best Parameters:", gs_cat_rus.best_params_)
print("Best Precision Score:", gs_cat_rus.best_score_)

tuned_cat_rus= gs_cat_rus.best_estimator_
y_pred_tuned_cat_rus= tuned_cat_rus.predict(X_test)
y_pred_prob_tuned_cat_rus= tuned_cat_rus.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_rus, labels= tuned_cat_rus.classes_)
print(classification_report(y_test, y_pred_tuned_cat_rus, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_rus, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_rus))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_rus))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6705170	total: 3.71ms	remaining: 367ms
1:	learn: 0.6505523	total: 8.24ms	remaining: 404ms
2:	learn: 0.6338121	total: 12.1ms	remaining: 393ms
3:	learn: 0.6195636	total: 17.6ms	remaining: 422ms
4:	learn: 0.6082679	total: 22.3ms	remaining: 423ms
5:	learn: 0.5979089	total: 26.5ms	remaining: 416ms
6:	learn: 0.5888106	total: 31.2ms	remaining: 414ms
7:	learn: 0.5814432	total: 36.5ms	remaining: 419ms
8:	learn: 0.5747500	total: 40.7ms	remaining: 411ms
9:	learn: 0.5683130	total: 45ms	remaining: 405ms
10:	learn: 0.5632037	total: 51.4ms	remaining: 416ms
11:	learn: 0.5583284	total: 56.1ms	remaining: 411ms
12:	learn: 0.5543527	total: 60.6ms	remaining: 405ms
13:	learn: 0.5503570	total: 66.8ms	remaining: 410ms
14:	learn: 0.5466976	total: 71.1ms	remaining: 403ms
15:	learn: 0.5434937	total: 75.9ms	remaining: 398ms
16:	learn: 0.5403217	total: 80.9ms	remaining: 395ms
17:	learn: 0.5384111	total: 85.5ms	remaining: 390ms
18:	learn: 0.5

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_rus, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_rus, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_rus, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_rus, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_rus, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_rus, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_rus, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_rus, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_rus, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_rus, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_rus, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_rus, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_rus, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_rus, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_rus, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_rus, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_rus, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_rus, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_rus, zero_division= 0))

Logistic Regression Precision: 0.24757443961191034
Decision Tree Precision: 0.16887510572314632
Tuned Decision Tree Precision: 0.21743325828240592
KNeighborsClassifier Precision: 0.20586400499064256
Tuned KNeighborsClassifier Precision: 0.22747252747252747
GaussianNB Precision: 0.1840074818798223
SVM Precision: 0.11044025157232705
Random Forest Precision: 0.22794117647058823
Tuned Random Forest Precision: 0.23636363636363636
AdaBoost Precision: 0.25624133148404993
Tuned AdaBoost Precision: 0.25624133148404993
GradientBoosting Precision: 0.24842245101295252
Tuned GradientBoosting Precision: 0.24842245101295252
XGB Precision: 0.22810913705583757
Tuned XGB Precision: 0.23630573248407644
LGBM Precision: 0.23419721871049304
Tuned LGBM Precision: 0.24104859335038364
CatBoost Precision: 0.23756377551020408
Tuned CatBoost Precision: 0.2520674826331459


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_rus, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_rus, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_rus, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_rus, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_rus, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_rus, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_rus, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_rus, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_rus, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_rus, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_rus, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_rus, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_rus, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_rus, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_rus, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_rus, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_rus, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_rus, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_rus, zero_division= 0)
                  }

rus_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
rus_precision= rus_precision.sort_values(by= 'Precision Score', ascending=False)
print(rus_precision)

                                    Model  Precision Score
9                     AdaBoost Precision:         0.256241
10              Tuned AdaBoost Precision:         0.256241
18              Tuned CatBoost Precision:         0.252067
11            GradientBoosting Precision:         0.248422
12      Tuned GradientBoosting Precision:         0.248422
0          Logistic Regression Precision:         0.247574
16                  Tuned LGBM Precision:         0.241049
17                    CatBoost Precision:         0.237564
8          Tuned Random Forest Precision:         0.236364
14                   Tuned XGB Precision:         0.236306
15                        LGBM Precision:         0.234197
13                         XGB Precision:         0.228109
7                Random Forest Precision:         0.227941
4   Tuned KNeighborsClassifier Precision:         0.227473
2          Tuned Decision Tree Precision:         0.217433
3         KNeighborsClassifier Precision:         0.2058

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_rus),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_rus),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_rus),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_rus),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_rus),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_rus),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_rus),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_rus),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_rus),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_rus),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_rus),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_rus),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_rus),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_rus),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_rus),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_rus),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_rus),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_rus),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_rus)
                  }

rus_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
rus_accuracy= rus_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(rus_accuracy)

                                   Model  Accuracy Score
9                     AdaBoost Accuracy:        0.754163
10              Tuned AdaBoost Accuracy:        0.754163
4   Tuned KNeighborsClassifier Accuracy:        0.745466
18              Tuned CatBoost Accuracy:        0.744300
0          Logistic Regression Accuracy:        0.743239
11            GradientBoosting Accuracy:        0.742603
12      Tuned GradientBoosting Accuracy:        0.742603
16                  Tuned LGBM Accuracy:        0.731467
17                    CatBoost Accuracy:        0.728709
14                   Tuned XGB Accuracy:        0.727649
15                        LGBM Accuracy:        0.724891
8          Tuned Random Forest Accuracy:        0.721816
13                         XGB Accuracy:        0.721498
2          Tuned Decision Tree Accuracy:        0.716937
7                Random Forest Accuracy:        0.714922
3         KNeighborsClassifier Accuracy:        0.703256
1                Decision Tree 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_rus),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_rus),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_rus),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_rus),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_rus),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_rus),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_rus),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_rus),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_rus),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_rus),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_rus),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_rus),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_rus),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_rus),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_rus),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_rus),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_rus),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_rus),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_rus)
                  }

rus_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
rus_auc= rus_auc.sort_values(by= 'AUC Score', ascending=False)
print(rus_auc)

                              Model  AUC Score
18              Tuned CatBoost AUC:   0.855030
10              Tuned AdaBoost AUC:   0.854098
9                     AdaBoost AUC:   0.854098
0          Logistic Regression AUC:   0.853814
11            GradientBoosting AUC:   0.851202
12      Tuned GradientBoosting AUC:   0.851202
16                  Tuned LGBM AUC:   0.849420
8          Tuned Random Forest AUC:   0.847230
17                    CatBoost AUC:   0.844735
15                        LGBM AUC:   0.843141
14                   Tuned XGB AUC:   0.842693
7                Random Forest AUC:   0.831485
13                         XGB AUC:   0.822483
5                   GaussianNB AUC:   0.805994
4   Tuned KNeighborsClassifier AUC:   0.797705
2          Tuned Decision Tree AUC:   0.795073
3         KNeighborsClassifier AUC:   0.776288
6                          SVM AUC:   0.698684
1                Decision Tree AUC:   0.655233
